In [ ]:
import bolsuite as bs
import pandas as pd
import numpy as np
import time
from datetime import date, timedelta, datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, LabelSet, TableColumn,DataTable
from IPython.display import clear_output

bsc = bs.Connector()

stock_list = [['GGAL', 'GGAL.O', 10], 
              ['YPFD', 'YPF.N', 1], 
              ['SUPV', 'SUPV.N', 5], 
              ['BBAR', 'BBAR.N', 3], 
              ['CEPU', 'CEPU.N', 10], 
              ['CRES', 'CRESY.O', 10],
              ['EDN', 'EDN.N', 20], 
              ['BMA', 'BMA.N', 10], 
              ['LOMA', 'LOMA.K', 5], 
              ['PAMP', 'PAM.N', 25], 
              ['TECO2', 'TEO.N', 5]]


intraday_CCL_list = pd.DataFrame(columns=['Time', 'CCL'])

result_list = pd.DataFrame(columns=['Local', 'Local Price', 'ADR', 'ADR Price', 'Ratio'])


def collect_CCL_data(local, adr, ratio):
    local_data = bsc.personal_portfolio(ticker=local)
    adr_data = bsc.adrs(ticker=adr)
    df = pd.DataFrame({'Local': [local], 'Local Price':[float(local_data['Close'].values)], 'ADR': [adr], 'ADR Price': [float(adr_data['Close'].values)], 'Ratio': ratio})
    return df

def ccl_calculation():
    global stock_list
    global result_list
    global intraday_CCL_list
    result_list = pd.DataFrame(columns=['Local', 'Local Price', 'ADR', 'ADR Price', 'Ratio'])

    for stock in stock_list:
        res = pd.DataFrame(collect_CCL_data(stock[0], stock[1], stock[2]))
        result_list = result_list.append(res, ignore_index=True)

    result_list['CCL'] = (result_list['Local Price'] * result_list['Ratio'] / result_list['ADR Price'])

    result_list['Average'] = result_list['CCL'].mean()
    result_list['Average'] = result_list['Average'].apply(lambda x:round(x,2))
    result_list['CCL'] = result_list['CCL'].apply(lambda x:round(x,2))
    
    intraday_CCL_list = intraday_CCL_list.append({'Time': datetime.now().strftime("%H:%M:%S"), 'CCL': result_list['Average'][0]}, ignore_index=True)
    intraday_CCL_list['CCL'] = intraday_CCL_list['CCL'].apply(lambda x:round(x,2))
        
    result_list.style
    output_graph()  
    
    time.sleep(30)

def output_graph():
    output_notebook()
    
    source = ColumnDataSource(result_list)
    p = figure(title="CCL", x_axis_label='Symbol', y_axis_label='CCL', x_range=result_list['Local'], plot_height=300)
    p.line(result_list['Local'], result_list['Average'], line_width=2)
    p.line(result_list['Local'], result_list['CCL'], line_width=2 , line_color="red")
    p.circle(result_list['Local'], result_list['CCL'], size=10 , fill_color="red")
    labels = LabelSet(x="Local", y="CCL", text="CCL", level='glyph', x_offset=7, y_offset=7, source=source, render_mode='canvas', text_font_size='10px')
    p.add_layout(labels)
    
    intraday_source = ColumnDataSource(intraday_CCL_list)
    a = figure(title="CCL Intraday", x_axis_label='Time', y_axis_label='CCL', x_range=intraday_CCL_list['Time'], plot_height=300)
    a.line(intraday_CCL_list['Time'], intraday_CCL_list['CCL'], line_width=2)
    a.circle(intraday_CCL_list['Time'], intraday_CCL_list['CCL'], size=5 , fill_color="red")
#     labels = LabelSet(x="Time", y="CCL", text="CCL", level='glyph', x_offset=7, y_offset=7, source=intraday_source, render_mode='canvas', text_font_size='9px')
    a.add_layout(labels)
    
    columns = [
        TableColumn(field="Local", title="Local"),
        TableColumn(field="Local Price", title="Local Price"),
        TableColumn(field="ADR", title="ADR"),
        TableColumn(field="ADR Price", title="ADR Price"),
        TableColumn(field="Ratio", title="Ratio"),
        TableColumn(field="CCL", title="CCL"),
        TableColumn(field="Average", title="Average")
    ]
    
    data_table = DataTable(source=source, columns=columns, width=500, height=400)
    
    grid = gridplot([[p, data_table], [None, a]])

    show(grid)

while True:
    for i in range(10):
        clear_output(wait=True)
    
    ccl_calculation()
    

Loading BokehJS ...